# Datenvorbereitung für ML-Modelle

Dieses Notebook erstellt den vollständigen Datensatz mit Wetter-, Kalender- und Ereignisfeatures, auf Basis von Umsatzdaten.  
Die Ausgabe ist eine `df_prepared.csv`, die für verschiedene Modelle (z. B. RF, NN) genutzt werden kann.

In [ ]:
import pandas as pd
import numpy as np
from itertools import product
from workalendar.europe import Germany

# CSVs laden
base_url = "https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/"
df_umsatz = pd.read_csv(base_url + "umsatzdaten_gekuerzt.csv")
df_wetter = pd.read_csv(base_url + "wetter.csv")
df_kiwo = pd.read_csv(base_url + "kiwo.csv")
df_ferien = pd.read_csv("../../data/ferien_sh.csv", parse_dates=["Startdatum", "Enddatum"])

In [ ]:
# Alle Datum-Warengruppe-Kombis
alle_daten = pd.date_range(start="2013-07-01", end="2019-07-31", freq="D")
warengruppen = [1, 2, 3, 4, 5, 6]
voll_kombis = pd.DataFrame(list(product(alle_daten, warengruppen)), columns=["Datum", "Warengruppe"])

# Wetterdaten mergen
df_wetter["Datum"] = pd.to_datetime(df_wetter["Datum"])
df = voll_kombis.merge(df_wetter, on="Datum", how="left")

# KiWo
df_kiwo["Datum"] = pd.to_datetime(df_kiwo["Datum"])
df_kiwo["KielerWoche"] = 1
df = df.merge(df_kiwo[["Datum", "KielerWoche"]], on="Datum", how="left").fillna({"KielerWoche": 0})

# Umsatzdaten
df_umsatz["Datum"] = pd.to_datetime(df_umsatz["Datum"])
df = df.merge(df_umsatz, on=["Datum", "Warengruppe"], how="left")
df["Umsatz"] = df["Umsatz"].fillna(0)

In [ ]:
# ID
df["id"] = df.apply(lambda row: int(row["Datum"].strftime("%y%m%d") + str(row["Warengruppe"])), axis=1)

# Kalenderfeatures
df["Wochentag"] = df["Datum"].dt.dayofweek
df["Monat"] = df["Datum"].dt.month
df["IstWochenende"] = df["Wochentag"].isin([5, 6]).astype(int)

# Feiertage (SH)
cal = Germany(subdivision="SH")
feiertage = set()
for jahr in df["Datum"].dt.year.unique():
    feiertage.update([d for d, _ in cal.holidays(jahr)])
df["Feiertag"] = df["Datum"].isin(feiertage).astype(int)

# Ferienzeit
ferien_tage = set()
for _, row in df_ferien.iterrows():
    ferien_tage.update(pd.date_range(row["Startdatum"], row["Enddatum"]))
df["Ferienzeit"] = df["Datum"].isin(ferien_tage).astype(int)

In [ ]:
# Auffüllen
df["Temperatur"] = df["Temperatur"].fillna(df["Temperatur"].median())
df["Bewoelkung"] = df["Bewoelkung"].fillna(df["Bewoelkung"].median())
df["Windgeschwindigkeit"] = df["Windgeschwindigkeit"].fillna(df["Windgeschwindigkeit"].median())
df["Wettercode"] = df["Wettercode"].fillna(-1).astype(int)

# Speichern
df.to_csv("../../data/df_prepared.csv", index=False)
print("✅ Datei gespeichert als df_prepared.csv")